# Python: Identify areas of potential foliage influence on railway network in autumn

In [ ]:
import geopandas as gpd
import shapely
import folium

## Add [WFS](https://python-gis-book.readthedocs.io/en/latest/part2/chapter-09/nb/01-retrieving-data-from-wfs.html) layer in GeoPandas

In [ ]:
wfs_url = 'https://geoserver.geonet-mrn.de/xdatatogo/db_strecken/ows?'\
            'service=WFS&'\
            'version=2.0.0&'\
            'request=GetFeature&'\
            'typename=xdatatogo:db_strecken&'\
            'outputFormat=json&'\
            'bbox=6.345854,51.295232,11.598078,54.13791,EPSG:4326'
railway_network = gpd.read_file(wfs_url)

Visualise downloaded data in map

In [ ]:
map = railway_network.explore(name='Railway network within bounding box')
folium.LayerControl().add_to(map)
map

## Reproject the extracted layer to [EPSG:3857](https://epsg.io/3857) in order to be able to use "metre" unit for buffering

In [ ]:
print("Current CRS:", railway_network.crs)

Change the projection

In [ ]:
railway_network = railway_network.to_crs(epsg=3857)
print("Projected CRS:", railway_network.crs)

## Create a buffer around the railway tracks

In [ ]:
buffer_distance_m = 200
# duplicate layer to separately store it with a buffered geometry
railway_network_buffer = railway_network
railway_network_buffer["geometry"] = railway_network["geometry"].buffer(buffer_distance_m)

Inspect the data table if it contains polygon geometries now

In [ ]:
railway_network_buffer.head()

## Identifying areas of foliage influence on railway network in autumn 

Load "Forst" dataset from a local Shapefile into a GeoDataFrame

In [ ]:
forst_patches = gpd.read_file('../data/DLM_250_Forst_NDS.shp')

Perform spatial [intersection](https://en.wikipedia.org/wiki/Vector_overlay#Operators) of our railway network with the forest patches

In [ ]:
result = gpd.overlay(railway_network_buffer, forst_patches, how='intersection')

## Plot the result

In [ ]:
map = result.explore(name='Susceptible railway areas')
folium.LayerControl().add_to(map)
map

## Optional: export result to a file

In [ ]:
output_file = 'railway_network_intersect_forest.json'
result.to_file(output_file)
print('File exported successfully.')